# FastPitch Speaker Interpolation

In this tutorial we will explore the following:
- What is speaker interpolation in a multi-speaker Text-to-Speech model
- Why we need speaker interpolation
- Step-by-step demo of performing speaker interpolation using pretrained multi-speaker FastPitch model

## License

> Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

### What is Speaker Interpolation in a Multi-speaker Text-to-speech Model

Given a Multi-speaker Text-to-Speech (TTS) model that can generate speech in more than one speaker's voice, Speaker Interpolation is the process of synthesizing new synthetic **voices by combining two or more existing speaker voices** without the need for any further finetuning. Before we go in depths of the process let's understand why we need speaker interpolation?

### Why we need speaker interpolation

To release any TTS model, trained on datasets where the speakers do not want speech to be generated in their voice, speaker interpolation is important. This technique can be used to replace the original speaker's in the pretrained model with the interpolated synthetic speakers. This work would solve this problem.


This technique can be used to generate large multi-speaker dataset for training Automatic Speech Recognition (ASR) models or voice conversion models. It enables users to create an infinite number of speaker voices. 

We can also use speaker interpolation for achieving varying multi-dialect or multi-emotional voices in text-to-speech (TTS) synthesis. They may
be used for personalizing speech and generating speech with specific voice characteristics in speech synthesis systems.

In case of self-supervised learning (SSL), speaker interpolation can be used for negative sampling. In addition to the above use cases, speaker interpolation can be used for data augmentation as well.

### Step-by-step demo of Speaker Interpolation using Pretrained multi-speaker FastPitch model

#### Method

1. Train a multi-speaker spectrogram generator model (like FastPitch) using embedding lookup table. This means that during training an embedding (speaker embedding) will be learnt for each speaker.

2. Extract speaker embeddings for any two speakers, you want to combine, from the model trained in step 1.

3. Perform a weighted sum of the two speaker embeddings extracted in step 2. This will give the speaker embedding for the interpolated speaker.

4. Use the speaker embedding obtained from step 3, to condition the multi-speaker model in step 1.

In [ ]:
# Install NeMo library. If you are running locally (rather than on Google Colab), comment out the below lines
# and instead follow the instructions at https://github.com/NVIDIA/NeMo#Installation
BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
# If seaborn is not installed already run this cell
!pip install seaborn

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""

import json

import torch
import numpy as np
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import seaborn as sns

from nemo.collections.tts.models import FastPitchModel, HifiGanModel

**Load Pretrained multi-speaker FastPitch model**

In [ ]:
# Let's view the available model checkpoints for FastPitch 
FastPitchModel.list_available_models()

In [ ]:
# We will choose the English multi-speaker model: tts_en_fastpitch_multispeaker
spec_gen_model = FastPitchModel.from_pretrained("tts_en_fastpitch_multispeaker")
spec_gen_model.cuda()

**Visualization**

To understand the process better, let's try to visualize how the embeddings get combined to create a new speaker embedding.

In [ ]:
# Original Speaker IDs
speaker_1 = 2
speaker_2 = 10

# Extracting speaker embeddings for speaker_1 and speaker_2
speaker_emb_1 = spec_gen_model.fastpitch.speaker_emb(torch.tensor(speaker_1, dtype=torch.int32).cuda()).clone().detach()
speaker_emb_2 = spec_gen_model.fastpitch.speaker_emb(torch.tensor(speaker_2, dtype=torch.int32).cuda()).clone().detach()

We will now visualize the two speaker embeddings. The embedding dimensions are 384, so to keep the visualization clear we will only visualize the first 30 elements of each embedding.

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 1]
tmp_emb = np.vstack([speaker_emb_1.cpu()[:30], speaker_emb_2.cpu()[:30]])
sns.heatmap(data=tmp_emb, cmap="coolwarm", xticklabels=False, yticklabels=["speaker1", "speaker2"])
plt.title("Visualizing Speaker Embeddings", fontsize =20)

Now let's combine the two speaker embeddings and generate the synthetic speaker embedding.

In [ ]:
# Weights assigned to each speaker, in the process of generating the new speaker
# You can play around with these weights
weight_speaker_1 = 0.5
weight_speaker_2 = 0.5

interpolated_speaker_emb = weight_speaker_1 * speaker_emb_1 + weight_speaker_2 * speaker_emb_2

We will now visualize the interpolated speaker along with the two speaker original embeddings

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 2]
tmp_emb = np.vstack([speaker_emb_1.cpu()[:30], speaker_emb_2.cpu()[:30], interpolated_speaker_emb.cpu()[:30]])
sns.heatmap(data=tmp_emb, cmap="coolwarm", xticklabels=False, yticklabels=["speaker1", "speaker2", "interpolated_speaker"])
plt.title("Visualizing Original and Interpolated Speaker Embeddings", fontsize =20)

**Extraction and Combining of speaker embeddings**

This entire process of doing speaker interpolation is performed by the method `interpolate_speaker` in the FastPitch class.

**Generate Speech using speaker interpolation**

Now we will use this process of speaker interpolation to generate speech using original speaker voices and interpolated speaker voice. For this we will need a vocoder to convert generated spectrogram to waveform speech. So let's load HiFiGAN vocoder.

In [ ]:
vocoder = HifiGanModel.from_pretrained("tts_en_hifitts_hifigan_ft_fastpitch")
vocoder.cuda().eval()

Now we will define a function which will take as input the spectrogram generator model, vocoder model, text to be converted to speech and speaker id. This function will return spectrogram and audio of the generated speech.

In [ ]:
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    spec_gen_model.eval()
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    else:
        raise Exception("None value was generated for spectrogram")
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

Now let's generate some speech.

In [ ]:
# Choose any original speakers (speaker1 and speaker2)
# Choose new_speaker ID such that it is not already assigned to an existing speaker
speaker1, speaker2, new_speaker = 6097, 92, 1000

# Choose weight1 corresponding to speaker1 and weight2 corresponding to speaker2
# You can change this and play around with the weights
weight1, weight2 = 0.4, 0.6

# Perform Speaker Interpolation
spec_gen_model.interpolate_speaker(speaker1, speaker2, weight1, weight2, new_speaker)

In [ ]:
# This is the text you want to convert to speech
text = "Hello World, Speaker Interpolation is cool"

_, audio = infer(spec_gen_model, vocoder, text, speaker=speaker1)
print(f"audio generated for speaker {speaker1}")
ipd.display(ipd.Audio(audio, rate=44100))

_, audio = infer(spec_gen_model, vocoder, text, speaker=speaker2)
print(f"audio generated for speaker {speaker2}")
ipd.display(ipd.Audio(audio, rate=44100))

_, audio = infer(spec_gen_model, vocoder, text, speaker=new_speaker)
print(f"audio generated for Interpolated Speaker")
ipd.display(ipd.Audio(audio, rate=44100))

It was observed that for speaker interpolation to work well and stable, the pretrained multi-speaker FastPitch model needs to be robust. So, training multi-speaker FastPitch model on larger data with larger number of speakers help in getting better interpolated speakers.